In [ ]:
import requests
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import re
import random
import time as t
from google.cloud import storage
from bs4 import BeautifulSoup
import os

# Main PD to GCP Command

In [ ]:
# takes the content and transfers to GCP 
# requires a json with api key
api_key_path = 'covid_news_scrape_gcp_key.json'
bucket = 'fan_project_news_stories'
counter = 0
nos = 0
folder = 'news_dfs_3.1_3.31'
for item in os.listdir(folder):
    news_df = pd.read_csv('{}/{}'.format(folder, item), index_col = 0, encoding='latin-1')
    if not news_df.empty and item not in domains_used:
        domain = news_df['source'][0].lower()
        if '.com' in domain or '.net' in domain:
            domain = re.sub("(.com)|(.net)", "", domain)
        domain = re.sub(" ", "_", domain)    

        def upload_text(gcp_api_key_path, bucket_name, filename, text):
            storage_client = storage.Client.from_service_account_json(gcp_api_key_path)
            bucket = storage_client.get_bucket(bucket_name)
            pathname = '{domain}/{filename}.txt'.format(domain = domain, filename = filename)
            if not bucket.blob(pathname).exists(storage_client):
                d = bucket.blob(pathname)
                d.upload_from_string(text)
            else:
                print('{} already exists in gcp; skipping'.format(filename))
                return
         
        # removes any video based links
        news_df_no_vids = news_df[~news_df['url'].str.contains('/video')]
        news_df_no_vids = news_df_no_vids[~news_df_no_vids['url'].str.contains('/radio')]
        news_df_no_vids = news_df_no_vids[~news_df_no_vids['url'].str.contains('/audio')]
        
        #print("{} na rate: {}".format(domain, news_df_no_vids['content'].isna().mean()))
        for index, row in news_df.iterrows():
            # construct the datetime metadata
            date = re.sub( '-', '.', row['publishedAt'].split("T",1)[0])
            time = re.sub(":", '.', row['publishedAt'].split("T",1)[1].split("Z",1)[0])

            source = re.sub("(.com)|(.net)", "", row['source'])
            metadata = '{}_{}.{}'.format(source, date, time)
            full_text = row['content']
            if type(full_text) is str and len(full_text) > 100:
                print('index: {} uploading document {} from df\n'.format(index, metadata))
                upload_text(api_key_path, bucket, '{metadata}.txt'.format(metadata=metadata), full_text)
            else:
                nos += 1
                print('index: {} document {} was empty and was skipped\n'.format(index, metadata))
            counter += 1
            t.sleep(random.uniform(0,1))
    domains_used.append(item)
print("total: {}\nNaNs: {}\nNaN Rate: {}".format(counter, nos, nos/counter) )

# Various other scraping cells